In [3]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize

In [4]:
import pandas as pd

# Load dataset
df = pd.read_csv(
    "ionosphere.data",
    header=None
)

# First 34 columns → features
X = df.iloc[:, :-1].values

# Last column → labels ('g' or 'b')
y_raw = df.iloc[:, -1].values

# Convert labels to 0 / 1
y = np.array([1 if label == 'g' else 0 for label in y_raw])


In [5]:
scaler = MinMaxScaler(feature_range=(0, np.pi))
X_scaled = scaler.fit_transform(X)

In [6]:
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X_scaled)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

In [8]:
def qnn_circuit(x, theta):
    qc = QuantumCircuit(2)
    qc.ry(x[0], 0)  #encode data
    qc.ry(x[1], 1)
    qc.ry(theta[0], 0) #tainable_parms
    qc.ry(theta[1], 1)
    qc.cx(0, 1) #entanglement
    qc.measure_all() #measurment
    return qc


In [9]:
backend = Aer.get_backend("qasm_simulator")

def predict(x, theta, shots=512):
    qc = qnn_circuit(x, theta)
    result = backend.run(qc, shots=shots).result()
    counts = result.get_counts()

    # Class 1 if qubit 0 = 1
    p1 = (counts.get('10', 0) + counts.get('11', 0)) / shots
    return p1

In [10]:
def loss(theta):
    total_loss = 0
    for i in range(len(X_train)):
        p = predict(X_train[i], theta)
        total_loss += (p - y_train[i])**2
    return total_loss / len(X_train)

In [11]:
theta_init = np.random.rand(2)

result = minimize(
    loss,
    theta_init,
    method="COBYLA",
    options={"maxiter": 50}
)

theta_opt = result.x
print("Optimized parameters:", theta_opt)

Optimized parameters: [0.64221025 1.1307886 ]


In [12]:
correct = 0

for i in range(len(X_test)):
    p = predict(X_test[i], theta_opt)
    y_pred = 1 if p >= 0.5 else 0

    if y_pred == y_test[i]:
        correct += 1

accuracy = correct / len(X_test)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.6901408450704225
